In [1]:
import torch
from torch import nn
from transformers import AutoTokenizer, AutoModel
import numpy as np

# Przykładowe dane
texts = [
    "Jestem bardzo szczęśliwy z dzisiejszego dnia.",
    "To był okropny tydzień, jestem załamany.",
    "Czuję się dobrze, choć trochę zmęczony.",
    "Jestem bardzo zły na to, co się stało."
]
labels = [1, 0, 1, 0]  # 1 = pozytywna emocja, 0 = negatywna

# Tokenizer + model
model_name = "dkleczek/bert-base-polish-cased-v1"
tokenizer = AutoTokenizer.from_pretrained(model_name)
bert_model = AutoModel.from_pretrained(model_name)

# Tokenizacja
tokens = tokenizer(
    texts,
    padding=True,
    truncation=True,
    max_length=64,
    return_tensors='pt'
)

# Pobranie embeddingów
with torch.no_grad():
    outputs = bert_model(**tokens)
    embeddings = outputs.last_hidden_state  # [batch, seq_len, hidden]

# Definicja sieci
class EmotionNet(nn.Module):
    def __init__(self, hidden_dim=768, num_classes=2):
        super(EmotionNet, self).__init__()
        self.lstm = nn.LSTM(hidden_dim, hidden_dim, batch_first=True, bidirectional=True)
        self.attn = nn.Sequential(
            nn.Linear(hidden_dim*2, 128),
            nn.Tanh(),
            nn.Linear(128, 1),
            nn.Softmax(dim=1)
        )
        self.conv = nn.Conv2d(1, 32, kernel_size=(3, hidden_dim*2))
        self.fc = nn.Linear(32, num_classes)

    def forward(self, x):
        lstm_out, _ = self.lstm(x)  # [batch, seq_len, hidden*2]
        attn_weights = self.attn(lstm_out)  # [batch, seq_len, 1]
        context = torch.sum(attn_weights * lstm_out, dim=1)  # [batch, hidden*2]
        conv_input = lstm_out.unsqueeze(1)  # [batch, 1, seq_len, hidden*2]
        conv_out = self.conv(conv_input).squeeze(3).max(2)[0]  # [batch, 32]
        out = self.fc(conv_out)
        return out

net = EmotionNet()
output = net(embeddings)
print(output)


tokenizer_config.json:   0%|          | 0.00/30.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/459 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/489k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/531M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/531M [00:00<?, ?B/s]

tensor([[0.1322, 0.1521],
        [0.1234, 0.2454],
        [0.1551, 0.1268],
        [0.1105, 0.1753]], grad_fn=<AddmmBackward0>)


In [2]:
class EmotionClassifierLSTM(nn.Module):
    def __init__(self, hidden_dim=768, num_classes=2):
        super(EmotionClassifierLSTM, self).__init__()
        self.lstm = nn.LSTM(hidden_dim, hidden_dim, bidirectional=True, batch_first=True)
        self.attn = nn.Sequential(
            nn.Linear(hidden_dim*2, 128),
            nn.Tanh(),
            nn.Linear(128, 1)
        )
        self.classifier = nn.Linear(hidden_dim*2, num_classes)

    def forward(self, x):
        lstm_out, _ = self.lstm(x)  # [batch, seq_len, hidden*2]
        attn_weights = torch.softmax(self.attn(lstm_out), dim=1)
        context = torch.sum(attn_weights * lstm_out, dim=1)  # [batch, hidden*2]
        return self.classifier(context)
